## 1. 라이브러리 Import
필요한 Python 라이브러리들을 가져옵니다. 웹 API 호출, 데이터 처리, 날짜 처리, 환경변수 관리 등에 사용됩니다.

In [1]:
# 라이브러리 import
import requests           # HTTP 요청을 위한 라이브러리 (Finnhub API 호출용)
import pandas as pd       # 데이터 분석 및 처리를 위한 라이브러리 (DataFrame 사용)
import datetime           # 날짜 및 시간 처리를 위한 라이브러리
import os                 # 운영체제 관련 기능 (환경변수 접근용)
from dotenv import load_dotenv  # .env 파일에서 환경변수 로드하는 라이브러리
import time               # 시간 지연 처리를 위한 라이브러리 (API 호출 제한 관리)

## 2. 환경 설정 로드
.env 파일에서 Finnhub API 키를 로드합니다. API 키는 보안상 코드에 직접 입력하지 않고 환경변수로 관리합니다.

In [2]:
# 환경 설정 로드
load_dotenv()  # 현재 디렉토리의 .env 파일에서 환경변수를 로드
# .env 파일에서 'finhub' 키로 저장된 Finnhub API 키를 가져옴
# 주의: 환경변수 이름이 'finnhub'가 아닌 'finhub'로 설정되어 있음
FINHUB_API_KEY = os.getenv("finhub")

## 3. API 호출 제한 설정
Finnhub API는 분당 호출 횟수에 제한이 있습니다. API 호출 간격을 조절하여 제한을 위반하지 않도록 설정합니다.

In [3]:
# API 호출 제한 설정
# Finnhub Free Tier는 분당 60회 호출 제한이 있음
API_CALLS_PER_MINUTE = 60

# 각 API 호출 간의 최소 대기 시간 계산 (초 단위)
# 60초 / 60회 = 1초 간격으로 호출하여 제한 내에서 안전하게 호출
DELAY_BETWEEN_CALLS = 60.0 / API_CALLS_PER_MINUTE

## 4. 안전한 datetime 변환 함수
Finnhub API에서 받은 timestamp를 pandas datetime으로 변환할 때 발생할 수 있는 오류를 방지하는 함수입니다.

In [4]:
# 안전한 datetime 변환 함수
def safe_datetime_conversion(timestamp):

    # 빈 값이나 0값 체크
    if not timestamp or timestamp == 0:
        return None
    
    try:
        # 유닉스 타임스탬프 범위 확인 (1970-01-01 이후만 허용)
        # 음수값은 1970년 이전을 의미하므로 제외
        if timestamp < 0:
            return None
            
        # pandas에서 처리 가능한 최대 timestamp 확인
        # 2262-04-11 이후는 pandas에서 overflow 발생
        if timestamp > 9223372036:  # 약 2262-04-11 23:47:16
            return None
            
        # Unix timestamp를 pandas datetime으로 변환
        # unit='s'는 초 단위임을 의미
        return pd.to_datetime(timestamp, unit='s')
        
    except (ValueError, OutOfBoundsDatetime, OverflowError):
        # pandas에서 발생하는 날짜 범위 오류 처리
        return None
    except Exception:
        # 기타 예상치 못한 오류 처리
        return None


## 5. Finnhub 뉴스 수집 함수 (메인 함수)
특정 기업의 뉴스를 대량으로 수집하는 핵심 함수입니다. 날짜 구간을 나누어 여러 번 API 호출하여 최대한 많은 뉴스를 수집합니다.

In [5]:
# Finnhub 뉴스 수집 함수 (대량 수집 최적화)
def fetch_finnhub_news_extended(symbol: str = "GOOGL", start_date: str = "2025-06-14", days_per_request: int = 30) -> pd.DataFrame:

    # API 키가 없으면 빈 DataFrame 반환
    if not FINHUB_API_KEY:
        return pd.DataFrame()

    # Finnhub Company News API 엔드포인트
    url = "https://finnhub.io/api/v1/company-news"
    
    try:
        # 현재 날짜 기준으로 수집 기간 설정
        today = datetime.date.today()
        
        # 3년 전부터 현재까지 수집 시도 (Free Tier 제한 테스트)
        three_years_ago = today - datetime.timedelta(days=1095)  # 1095일 = 약 3년
        actual_start = three_years_ago  # 실제 수집 시작 날짜
        actual_end = today             # 실제 수집 종료 날짜
        
    except ValueError:
        # 날짜 처리 오류시 빈 DataFrame 반환
        return pd.DataFrame()

    # 모든 수집된 기사를 저장할 리스트
    all_articles = []
    # 현재 처리 중인 날짜
    current_date = actual_start
    # API 호출 횟수 카운터
    request_count = 0
    
    # 시작 날짜부터 종료 날짜까지 반복 처리
    while current_date < actual_end:
        # 각 요청의 종료 날짜 계산 (지정된 일수만큼 또는 전체 종료 날짜까지)
        period_end = min(current_date + datetime.timedelta(days=days_per_request), actual_end)
        
        # API 요청 파라미터 설정
        params = {
            "symbol": symbol,                           # 주식 심볼
            "from": current_date.isoformat(),           # 시작 날짜 (YYYY-MM-DD)
            "to": period_end.isoformat(),               # 종료 날짜 (YYYY-MM-DD)
            "token": FINHUB_API_KEY                     # API 인증 토큰
        }
        
        try:
            request_count += 1  # API 호출 횟수 증가
            
            # API 호출 제한을 위한 딜레이 (분당 60회 제한 준수)
            time.sleep(DELAY_BETWEEN_CALLS)
            
            # HTTP GET 요청으로 뉴스 데이터 가져오기
            res = requests.get(url, params=params, timeout=30)
            
            # HTTP 429 (Too Many Requests) 오류 처리
            if res.status_code == 429:
                time.sleep(10)  # 10초 대기 후 재시도
                res = requests.get(url, params=params, timeout=30)
            
            # HTTP 403 (Forbidden) 오류시 수집 중단 (API 제한 도달)
            if res.status_code == 403:
                break
            
            # 기타 HTTP 오류 처리
            if res.status_code != 200:
                if res.status_code != 429:
                    # 429가 아닌 오류는 다음 기간으로 건너뛰기
                    current_date = period_end + datetime.timedelta(days=1)
                    continue
                else:
                    # 429 오류는 30초 대기 후 재시도
                    time.sleep(30)
                    continue

            # JSON 응답 데이터 파싱
            data = res.json()
            
            # 응답이 리스트가 아닌 경우 (오류 응답 등) 처리
            if not isinstance(data, list):
                if isinstance(data, dict) and 'error' in data:
                    # API 제한 관련 오류 메시지 확인
                    if 'limit' in data['error'].lower():
                        break  # 제한 도달시 수집 중단
                # 다음 기간으로 이동
                current_date = period_end + datetime.timedelta(days=1)
                continue

            # 해당 기간의 뉴스 데이터 처리
            period_articles = []
            for item in data:
                # API에서 받은 timestamp를 안전하게 datetime으로 변환
                pub_date = safe_datetime_conversion(item.get("datetime"))
                
                # 뉴스 기사 정보를 딕셔너리로 구성
                article = {
                    "id": item.get("id"),                           # 기사 고유 ID
                    "title": item.get("headline", ""),             # 기사 제목
                    "summary": item.get("summary", ""),            # 기사 요약
                    "link": item.get("url", ""),                   # 기사 URL
                    "publisher": item.get("publisher", ""),        # 발행사
                    "category": item.get("category", ""),          # 카테고리
                    "pubDate": pub_date,                           # 발행 날짜
                    "image": item.get("image", ""),                # 이미지 URL
                    "related": item.get("related", ""),            # 관련 정보
                    "source": item.get("source", ""),              # 소스
                    "collection_period": f"{current_date.isoformat()}_{period_end.isoformat()}"  # 수집 기간 정보
                }
                period_articles.append(article)
            
            # 현재 기간의 기사들을 전체 리스트에 추가
            all_articles.extend(period_articles)
            
        except Exception as e:
            # 예외 발생시 API 제한 관련 오류인지 확인
            if "limit" in str(e).lower() or "403" in str(e):
                break  # API 제한 관련 오류시 수집 중단
        
        # 다음 기간으로 이동 (하루 간격으로 겹치지 않게)
        current_date = period_end + datetime.timedelta(days=1)
        
        # API 호출 제한 방지를 위한 추가 대기
        # 10회 호출마다 2초 추가 대기로 안전성 확보
        if request_count % 10 == 0:
            time.sleep(2)

    # 수집된 모든 기사 데이터 처리
    if all_articles:
        # 리스트를 pandas DataFrame으로 변환
        df = pd.DataFrame(all_articles)
        
        # 중복 기사 제거 (ID, 제목, 링크 기준으로 중복 판별)
        df = df.drop_duplicates(subset=['id', 'title', 'link'])
        
        # 날짜순으로 정렬 (최신 기사가 위에 오도록)
        # na_position='last'로 날짜가 없는 기사는 맨 아래로
        df = df.sort_values('pubDate', ascending=False, na_position='last')
        
        return df
    else:
        # 수집된 기사가 없으면 빈 DataFrame 반환
        return pd.DataFrame()


## 6. 수집 설정값 정의
뉴스 수집을 위한 기본 설정값들을 정의합니다. 필요에 따라 이 값들을 수정하여 다른 기업이나 기간의 뉴스를 수집할 수 있습니다.

In [6]:
# 뉴스 수집을 위한 설정값 정의
# 수집 대상 기업의 주식 심볼 (미국 상장 기업)
# 예시: "AAPL"(Apple), "GOOGL"(Google), "TSLA"(Tesla), "MSFT"(Microsoft), "AMZN"(Amazon) 등
target_symbol = "GOOGL"

# 참고용 시작 날짜 (실제로는 3년 전부터 자동으로 수집됨)
# YYYY-MM-DD 형식으로 입력
start_date = "2025-06-13"

# 한 번의 API 호출당 수집할 일수 설정
# 값이 작을수록 더 많은 API 호출을 하게 되어 더 많은 뉴스를 수집할 수 있음
# 하지만 API 제한에 도달할 가능성도 높아짐
days_per_request = 7

## 1-7. 뉴스 데이터 수집 실행
위에서 정의한 설정값들을 사용하여 실제로 뉴스 데이터를 수집합니다. 이 과정에서 API 호출이 여러 번 발생하므로 시간이 걸릴 수 있습니다.


In [7]:
# 뉴스 데이터 수집 실행
# 위에서 정의한 함수와 설정값을 사용하여 뉴스 수집을 시작
# 이 과정은 API 호출 제한으로 인해 시간이 걸릴 수 있음 (몇 분에서 몇십 분)
df_extended_news = fetch_finnhub_news_extended(
    symbol=target_symbol,           # 수집할 기업의 주식 심볼
    start_date=start_date,          # 참고용 시작 날짜
    days_per_request=days_per_request  # 한 번에 수집할 일수
)

## 8. 수집 결과 확인 및 기본 분석
수집된 뉴스 데이터의 기본적인 통계 정보를 확인합니다. 총 기사 수, 수집 기간, 유효한 날짜 정보 등을 출력합니다.


In [8]:
# 수집 결과 확인 및 기본 통계 분석
# DataFrame이 비어있지 않은지 확인 (수집에 성공했는지 체크)
if not df_extended_news.empty:
    # 총 수집된 기사 수 출력
    print(f"총 수집 기사 수: {len(df_extended_news)}개")
    
    # 날짜별 기사 분포 분석
    if 'pubDate' in df_extended_news.columns:
        # 유효한 날짜 정보가 있는 기사들만 필터링
        valid_dates = df_extended_news[df_extended_news['pubDate'].notna()].copy()
        
        if not valid_dates.empty:
            # 날짜만 추출 (시간 정보 제거)
            valid_dates['date_only'] = valid_dates['pubDate'].dt.date
            
            # 날짜별 기사 수 집계 및 정렬
            date_counts = valid_dates['date_only'].value_counts().sort_index()
            
            # 수집 기간 (가장 오래된 날짜 ~ 가장 최근 날짜) 출력
            print(f"수집 기간: {date_counts.index.min()} ~ {date_counts.index.max()}")
            
            # 유효한 날짜가 있는 기사 수와 전체 기사 수 비교
            print(f"유효한 날짜 기사: {len(valid_dates)}개 / 전체 {len(df_extended_news)}개")
else:
    # 수집에 실패한 경우 오류 메시지 출력
    print("뉴스 수집에 실패했습니다.")


총 수집 기사 수: 8563개
수집 기간: 2024-06-24 ~ 2025-06-18
유효한 날짜 기사: 8561개 / 전체 8563개


## 9. 데이터 미리보기
수집된 뉴스 데이터의 일부를 표 형태로 확인합니다. 제목, 발행처, 발행 날짜 등 주요 정보를 미리 볼 수 있습니다.


In [9]:
# 수집된 뉴스 데이터 미리보기
# 데이터가 정상적으로 수집되었는지 확인
if not df_extended_news.empty:
    print("최신 뉴스 미리보기:")
    
    # 주요 컬럼들만 선택하여 상위 5개 기사 표시
    # title: 기사 제목, publisher: 발행처, pubDate: 발행 날짜
    preview_data = df_extended_news[['title', 'publisher', 'pubDate']].head()
    
    # Jupyter Notebook에서 표 형태로 깔끔하게 출력
    display(preview_data)


최신 뉴스 미리보기:


,title,publisher,pubDate
8369,"Amazon's Zoox, Alphabet's Waymo in NYC, Oracle...",,2025-06-18 17:16:46
8370,Waymo applies for special permit to bring its ...,,2025-06-18 17:03:40
8371,Waymo has set its robotaxi sights on NYC,,2025-06-18 16:59:04
8373,Uber and Lyft stock fall after Waymo applies f...,,2025-06-18 16:44:50
8372,"VW to sell ID. Buzz robotaxis next year, takin...",,2025-06-18 16:19:53


## 10. CSV 파일로 저장
수집된 뉴스 데이터를 CSV 파일로 저장합니다. 파일명에는 기업 심볼과 현재 날짜가 포함되어 구분하기 쉽게 만듭니다.

In [10]:
# 수집된 뉴스 데이터를 CSV 파일로 저장
# 데이터가 있는 경우에만 저장 진행
if not df_extended_news.empty:
    # 현재 날짜를 ISO 형식 (YYYY-MM-DD)으로 가져오기
    today = datetime.date.today().isoformat()
    
    # 파일명 생성: "기업심볼_extended_news_날짜.csv" 형식
    # 예: "GOOGL_extended_news_2025-06-19.csv"
    filename = f"{target_symbol}_extended_news_{today}.csv"
    
    # DataFrame을 CSV 파일로 저장
    # index=False: 행 번호를 파일에 포함하지 않음
    # encoding='utf-8-sig': 한글 등 유니코드 문자가 깨지지 않도록 설정
    df_extended_news.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 저장 완료 메시지 출력
    print(f"파일 저장 완료: {filename}")
else:
    # 데이터가 없는 경우 경고 메시지 출력
    print("저장할 데이터가 없습니다.")


파일 저장 완료: GOOGL_extended_news_2025-06-19.csv
